# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [15]:
# Make a reference to the city file path
csv_path = "output_data/cities.csv"

# Import the books.csv file as a DataFrame
cities_df = pd.read_csv(csv_path, encoding="utf-8").dropna()
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Arraial do Cabo,-22.9661,-42.0278,76.91,85,0,25.64,BR,1612494174
1,Khatanga,71.9667,102.5000,-30.86,78,0,9.26,RU,1612494274
2,Qaanaaq,77.4840,-69.3632,-18.67,63,61,8.05,GL,1612494275
3,Matagami,49.7502,-77.6328,24.80,74,90,10.36,CA,1612494275
4,Kapaa,22.0752,-159.3190,70.00,56,90,5.75,US,1612494063


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [16]:
gmaps.configure(g_key)

In [17]:
loc = cities_df[['Lat','Lng']]
humidity = cities_df['Humidity']
fig = gmaps.figure(center=(46,-5), zoom_level=2)
heat_layer=gmaps.heatmap_layer(loc,weights=humidity, dissipating=False, max_intensity=300, point_radius=2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [26]:
df = cities_df[(cities_df['Max Temp']>69)&(cities_df['Max Temp']<81)&(cities_df['Wind Speed']<10)&(cities_df.Cloudiness==0)]
df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
103,Comodoro Rivadavia,-45.8667,-67.5000,73.40,56,0,3.44,AR,1612494281
207,Neuquén,-38.9516,-68.0591,75.20,57,0,2.30,AR,1612494493
251,Jizan,17.3333,42.6667,71.60,73,0,9.22,SA,1612494500
308,Tezu,27.9167,96.1667,74.39,19,0,1.72,IN,1612494506
337,Esim,4.8699,-2.2405,79.09,88,0,7.31,GH,1612494510
340,Mandera,3.9366,41.8670,69.69,82,0,6.73,KE,1612494510
348,Luorong,24.4058,109.6086,74.55,37,0,7.23,CN,1612494511
378,Natal,-5.7950,-35.2094,78.80,78,0,5.75,BR,1612494446
387,Tonj,7.2737,28.6822,73.58,21,0,6.55,SS,1612494516
390,Tura,25.5198,90.2201,71.76,43,0,1.10,IN,1612494516


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [41]:
hotel_df = df[['City','Country','Lat','Lng']].copy().reset_index().drop(columns='index')
hotel_df['Hotel Name'] = ''
hotel_df
#.drop(columns='index')

,City,Country,Lat,Lng,Hotel Name
0,Comodoro Rivadavia,AR,-45.8667,-67.5000,
1,Neuquén,AR,-38.9516,-68.0591,
2,Jizan,SA,17.3333,42.6667,
3,Tezu,IN,27.9167,96.1667,
4,Esim,GH,4.8699,-2.2405,
5,Mandera,KE,3.9366,41.8670,
6,Luorong,CN,24.4058,109.6086,
7,Natal,BR,-5.7950,-35.2094,
8,Tonj,SS,7.2737,28.6822,
9,Tura,IN,25.5198,90.2201,


In [42]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {'type' : 'hotel',
         'keyword' : 'hotel',
         'radius' : 5000,
         'key' : g_key
}


In [43]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"

    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 0: Comodoro Rivadavia.
Closest hotel in Comodoro Rivadavia is Lucania Palazzo Hotel.
------------
Retrieving Results for Index 1: Neuquén.
Closest hotel in Neuquén is Hilton Garden Inn Neuquen.
------------
Retrieving Results for Index 2: Jizan.
Missing field/result... skipping.
------------
Retrieving Results for Index 3: Tezu.
Closest hotel in Tezu is Oshin Hotel.
------------
Retrieving Results for Index 4: Esim.
Closest hotel in Esim is Ankobra Beach.
------------
Retrieving Results for Index 5: Mandera.
Closest hotel in Mandera is Granada hotel Mandera.
------------
Retrieving Results for Index 6: Luorong.
Missing field/result... skipping.
------------
Retrieving Results for Index 7: Natal.
Closest hotel in Natal is eSuites VILA DO MAR Natal.
------------
Retrieving Results for Index 8: Tonj.
Missing field/result... skipping.
------------
Retrieving Results for Index 9: Tura.
Closest hotel in Tura is Hotel Polo Orchid.
------------
Retrieving Results f

In [44]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [45]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))